In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-preview.16"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.16"
#r "nuget: FSharp.Stats, 0.4.1"
#r "nuget: FSharpAux"

Installed Packages Deedle, 2.3.0 FSharp.Stats, 0.4.1 FSharpAux, 1.1.0 Plotly.NET, 2.0.0-preview.16 Plotly.NET.Interactive, 2.0.0-preview.16

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
open Deedle
open Plotly.NET
open Plotly.NET.LayoutObjects
open FSharp.Stats
open System.IO
open FSharpAux

let readQuantAndProtFrame (path: string) =
    let frame =
        Frame.ReadCsv (path,true,separators="\t")
        |> Frame.indexRowsUsing (fun s ->
            {|
                ProteinGroup = 
                    s.GetAs<string>("ProteinGroup")
                    |> String.split ';'
                    |> Array.sort
                    |> String.concat ";"
                StringSequence = s.GetAs<string>("StringSequence")
                PepSequenceID = s.GetAs<int>("PepSequenceID")
                Charge = s.GetAs<int>("Charge")
                Synonym = s.GetAs<string>("Synonym")
            |}
        )
    frame

let data = readQuantAndProtFrame "../runs/VP21_WC/WCAnnotated.txt"

In [ ]:
#r "nuget: BIO-BTE-06-L-7_Aux, 0.0.9"

Installed Packages BIO-BTE-06-L-7_Aux, 0.0.9

In [ ]:
open BIO_BTE_06_L_7_Aux.Deedle_Aux

let colNames = [
    "WCGr1_U1"
    "WCGr1_U2"
    "WCGr1_U3"
    "WCGr1_U4"
    "WCGr2_U1"
    "WCGr2_U2"
    "WCGr2_U3"
    "WCGr2_U4"
    "WCGr3_U1"
    "WCGr3_U2"
    "WCGr3_U3"
    "WCGr3_U4"
]

type Qprot =
    | CBB
    | PS

let sliceQuantColumns quantColID =
    data
    |> Frame.mapRowKeys (fun k ->
        let qprot =
            match k.ProteinGroup |> String.contains "QProt_newCBB", k.ProteinGroup |> String.contains "QProt_newPS" with
            | true, false  -> Some CBB
            | false, true  -> Some PS
            | _ -> None  
        {|k with QProt = qprot|}
        )
    |> Frame.filterRows (fun k s -> k.QProt.IsSome)
    |> Frame.mapRowKeys (fun k -> k.ProteinGroup, k.StringSequence, k.PepSequenceID, k.Charge, k.Synonym)
    |> Frame.filterCols (fun ck os -> ck |> String.contains ("." + quantColID))
    |> Frame.mapColKeys (fun ck -> ck.Split('.') |> Array.item 0)
    |> Frame.sliceCols colNames

sliceQuantColumns "Ratio"
|> fun f -> f.SaveCsv("../runs/VP21_WC/WCAnnotated_Ratio.txt", true, ["ProteinGroup"; "StringSequence"; "PepSequenceID"; "Charge"; "Synonym"], '\t')
// /// Display the first 10 rows of the frame
// |> Frame.take 10
// |> formatAsTable 500
// |> Chart.withTemplate ChartTemplates.dark

